In [1]:
%pylab inline
from numpy.lib.recfunctions import append_fields


Populating the interactive namespace from numpy and matplotlib


In [2]:
### Excluding the AGB stars with dust spheres around.
x = np.load("../data/GDR3/gaiadr3_0ext.npy")
cut = (x['log_lum'] <5.) & (x['log_lum'] >3.) & (x['log_teff'] <3.7) & (x['gaia_g'] > 0.5)
print(len(x), len(x[cut]))

8102858 322023


In [3]:
x = np.load("../data/UBV/ubv_0ext.npy")
print(x.dtype.names)

('meh_ini', 'log_age', 'm_ini', 'm_act', 'log_lum', 'log_teff', 'log_grav', 'ubv_u', 'ubv_b', 'ubv_v', 'ubv_r', 'ubv_i', 'ubv_j', 'ubv_h', 'ubv_k', 'ubv_l', 'ubv_m')


In [4]:
avs = np.array([0,1,2,3,5,10,20])
x = []
for av in avs:
    temp = np.load("../data/UBV/ubv_%dext.npy" %(av))
    # we exclude the AGB dust stars from the averaging
    temp = temp[~cut]
    temp = append_fields(temp,"parsec_index",np.zeros(shape = len(temp),dtype = np.int32),usemask=False)
    x.append(temp)
print(len(x[0]),x[0].dtype)

7780835 [('meh_ini', '<f8'), ('log_age', '<f8'), ('m_ini', '<f8'), ('m_act', '<f8'), ('log_lum', '<f8'), ('log_teff', '<f8'), ('log_grav', '<f8'), ('ubv_u', '<f8'), ('ubv_b', '<f8'), ('ubv_v', '<f8'), ('ubv_r', '<f8'), ('ubv_i', '<f8'), ('ubv_j', '<f8'), ('ubv_h', '<f8'), ('ubv_k', '<f8'), ('ubv_l', '<f8'), ('ubv_m', '<f8'), ('parsec_index', '<i4')]


In [5]:
for name in x[0].dtype.names:
    print(name,x[0][name],len(np.unique(x[0][name])))

meh_ini [-1.49213 -1.49213 -1.49213 ...  0.33631  0.33631  0.33631] 75
log_age [ 6.6      6.6      6.6     ... 10.12004 10.12004 10.12004] 177
m_ini [0.09       0.0956148  0.1        ... 5.80000019 5.99999952 6.19999933] 6067091
m_act [0.09  0.096 0.1   ... 0.94  0.961 0.962] 53226
log_lum [-1.531 -1.502 -1.481 ... -4.601 -4.603 -4.604] 10836
log_teff [3.5276 3.5305 3.5326 ... 3.6298 3.6321 3.6319] 25411
log_grav [3.986 3.995 4.001 ... 8.484 8.505 8.505] 11069
ubv_u [13.125 12.983 12.885 ... 19.668 19.648 19.652] 33262
ubv_b [11.713 11.594 11.511 ... 18.373 18.353 18.357] 30578
ubv_v [10.087  9.985  9.911 ... 17.089 17.076 17.079] 28847
ubv_r [ 8.972  8.88   8.813 ... 16.223 16.215 16.218] 27352
ubv_i [ 7.977  7.902  7.847 ... 15.521 15.524 15.526] 26990
ubv_j [ 6.845  6.781  6.733 ... 14.712 14.722 14.723] 26750
ubv_h [ 6.263  6.196  6.146 ... 14.107 14.118 14.12 ] 26611
ubv_k [ 6.031  5.966  5.917 ... 13.964 13.979 13.98 ] 26512
ubv_l [ 5.754  5.695  5.65  ... 13.86  13.878 13.879] 2

In [6]:
def return_index_feh(feh):
        dfeh = 0.05
        offset = 1.5
        return(int((feh+offset)/dfeh))
min_value = min(x[0]['meh_ini'])
max_value = max(x[0]['meh_ini'])
min_index_feh = return_index_feh(min_value)
max_index_feh = return_index_feh(max_value)
stretch = max_index_feh - min_index_feh
print('teff values in parsec')
print('teff min max value: ', min_value, max_value )
print('teff min max index: ', min_index_feh, max_index_feh)
print('dimension cut into %d pieces' %(stretch))

def return_index_teff(teff):
        dteff = 0.02
        return(int(teff/dteff))
min_value = min(x[0]['log_teff'])
max_value = max(x[0]['log_teff'])
min_index_teff = return_index_teff(min_value)
max_index_teff = return_index_teff(max_value)
stretch = max_index_teff - min_index_teff
print('teff values in parsec')
print('teff min max value: ', min_value, max_value )
print('teff min max index: ', min_index_teff, max_index_teff)
print('dimension cut into %d pieces' %(stretch))

def return_index_lum(lum):
        dlum = 0.05
        offset = 5.
        return(int((lum+offset)/dlum))

min_value = min(x[0]['log_lum'])
max_value = max(x[0]['log_lum'])
min_index_lum = return_index_lum(min_value)
max_index_lum = return_index_lum(max_value)
stretch = max_index_lum - min_index_lum
print('lum values in parsec')
print('lum min max value: ', min_value, max_value )
print('lum min max index: ', min_index_lum, max_index_lum )
print('dimension cut into %d pieces' %(stretch))

def return_index(feh,teff,lum):
        """
        feh given in dex
        teff given in log teff
        lum given in log lum
        """
        index_feh = return_index_feh(feh)
        if index_feh < min_index_feh:
            index_feh = min_index_feh
        elif index_feh > max_index_feh:
            index_feh = max_index_feh

        index_teff = return_index_teff(teff)
        if index_teff < min_index_teff:
            index_teff = min_index_teff
        elif index_teff > max_index_teff:
            index_teff = max_index_teff
        index_teff *= 1000

        index_lum = return_index_lum(lum)
        if index_lum > max_index_lum:
            index_lum = max_index_lum
        elif index_lum < min_index_lum:
            index_lum = min_index_lum
        index_lum *= 1000 * 1000
        assert(index_feh >= 0)
        assert(index_teff >= 0)
        assert(index_lum >= 0)
        return (index_feh + index_teff + index_lum)

teff values in parsec
teff min max value:  -1.49213 0.33631
teff min max index:  0 36
dimension cut into 36 pieces
teff values in parsec
teff min max value:  2.4484 5.6843
teff min max index:  122 284
dimension cut into 162 pieces
lum values in parsec
lum min max value:  -4.604 6.24
lum min max index:  7 224
dimension cut into 217 pieces


In [7]:
print('indexing isochrones')
for i in range(len(x[0])):
    x[0]["parsec_index"][i] = return_index(x[0]["meh_ini"][i],x[0]["log_teff"][i],x[0]["log_lum"][i])

indexing isochrones


In [8]:
# All need to be sorted by parsec_index
sort_ind = np.argsort(x[0]['parsec_index'], kind = 'mergesort')
for i in range(len(avs)):
    x[i] = x[i][sort_ind]

In [9]:
u_ext = np.zeros(shape=(len(x[0]),len(x)))
b_ext = np.zeros(shape=(len(x[0]),len(x)))
v_ext = np.zeros(shape=(len(x[0]),len(x)))
r_ext = np.zeros(shape=(len(x[0]),len(x)))
i_ext = np.zeros(shape=(len(x[0]),len(x)))
j_ext = np.zeros(shape=(len(x[0]),len(x)))
h_ext = np.zeros(shape=(len(x[0]),len(x)))
k_ext = np.zeros(shape=(len(x[0]),len(x)))
l_ext = np.zeros(shape=(len(x[0]),len(x)))
m_ext = np.zeros(shape=(len(x[0]),len(x)))

for i,item in enumerate(x):
    print(i,item[0])
    u_ext[:,i] = item["ubv_u"]
    b_ext[:,i] = item["ubv_b"]
    v_ext[:,i] = item["ubv_v"]
    r_ext[:,i] = item["ubv_r"]
    i_ext[:,i] = item["ubv_i"]
    j_ext[:,i] = item["ubv_j"]
    h_ext[:,i] = item["ubv_h"]
    k_ext[:,i] = item["ubv_k"]
    l_ext[:,i] = item["ubv_l"]
    m_ext[:,i] = item["ubv_m"]

0 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 19.002, 18.082, 16.94, 16.147, 15.53, 14.762, 14.198, 14.048, 13.931, 13.92, 7181017)
1 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 20.539, 19.357, 17.928, 16.946, 16.128, 15.052, 14.381, 14.163, 13.986, 13.953, 0)
2 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 22.068, 20.622, 18.91, 17.735, 16.723, 15.341, 14.564, 14.277, 14.041, 13.987, 0)
3 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 23.583, 21.867, 19.879, 18.505, 17.309, 15.629, 14.746, 14.392, 14.096, 14.02, 0)
4 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 26.613, 24.359, 21.818, 20.045, 18.482, 16.204, 15.111, 14.62, 14.205, 14.086, 0)
5 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 34.081, 30.39, 26.551, 23.692, 21.341, 17.628, 16.019, 15.19, 14.48, 14.253, 0)
6 (-0.625, 10.12004, 5.1476984, 0.979, -4.601, 3.6351, 8.522, 48.775, 41.797, 35.636, 30.258, 26.828, 20.426, 17.818, 16.322, 15.027,

In [10]:
# show the extinction law and on which parameters it is dependent
u_ext = u_ext - u_ext[:,0,None]
b_ext = b_ext - b_ext[:,0,None]
v_ext = v_ext - v_ext[:,0,None]
r_ext = r_ext - r_ext[:,0,None]
i_ext = i_ext - i_ext[:,0,None]
j_ext = j_ext - j_ext[:,0,None]
h_ext = h_ext - h_ext[:,0,None]
k_ext = k_ext - k_ext[:,0,None]
l_ext = l_ext - l_ext[:,0,None]
m_ext = m_ext - m_ext[:,0,None]

In [11]:
ext = np.zeros(shape = (len(np.unique(x[0]["parsec_index"])),10,6),dtype = np.float32)
ext_std = np.zeros_like(ext)
parsec = np.zeros(shape =  (len(np.unique(x[0]["parsec_index"]))),dtype = x[0].dtype)
parsec_std = np.zeros(shape =  (len(np.unique(x[0]["parsec_index"]))),dtype = x[0].dtype)

In [12]:
parsec["parsec_index"] = np.unique(x[0]["parsec_index"])
parsec_std["parsec_index"] = np.unique(x[0]["parsec_index"])

In [13]:
indexing = np.hstack((0,np.searchsorted(x[0]['parsec_index'],parsec['parsec_index'], side='right')))
print(indexing)
for i in range(len(indexing)-1):
    if i%10000==0:
        print(i, len(indexing))
    lower = indexing[i]
    upper = indexing[i+1]
    #parsec['parsec_index'][i]
    #x[0]['parsec_index'][lower:upper]
    temp = x[0][lower:upper]
    u_temp = u_ext[lower:upper]
    b_temp = b_ext[lower:upper]
    v_temp = v_ext[lower:upper]
    r_temp = r_ext[lower:upper]
    i_temp = i_ext[lower:upper]
    j_temp = j_ext[lower:upper]
    h_temp = h_ext[lower:upper]
    k_temp = k_ext[lower:upper]
    l_temp = l_ext[lower:upper]
    m_temp = m_ext[lower:upper]
    for jtem in parsec.dtype.names:
        if jtem == "parsec_index":
            assert(parsec["parsec_index"][i] == temp["parsec_index"][0])
        else:
            parsec[jtem][i] = np.median(temp[jtem])
            parsec_std[jtem][i] = np.std(temp[jtem])
    for t in np.arange(6):
        ext[i,0,t] = np.median(u_temp[:,t+1])
        ext[i,1,t] = np.median(b_temp[:,t+1])
        ext[i,2,t] = np.median(v_temp[:,t+1])
        ext[i,3,t] = np.median(r_temp[:,t+1])
        ext[i,4,t] = np.median(i_temp[:,t+1])
        ext[i,5,t] = np.median(j_temp[:,t+1])
        ext[i,6,t] = np.median(h_temp[:,t+1])
        ext[i,7,t] = np.median(k_temp[:,t+1])
        ext[i,8,t] = np.median(l_temp[:,t+1])
        ext[i,9,t] = np.median(m_temp[:,t+1])
        
        ext_std[i,0,t] = np.std(u_temp[:,t+1])
        ext_std[i,1,t] = np.std(b_temp[:,t+1])
        ext_std[i,2,t] = np.std(v_temp[:,t+1])
        ext_std[i,3,t] = np.std(r_temp[:,t+1])
        ext_std[i,4,t] = np.std(i_temp[:,t+1])
        ext_std[i,5,t] = np.std(j_temp[:,t+1])
        ext_std[i,6,t] = np.std(h_temp[:,t+1])
        ext_std[i,7,t] = np.std(k_temp[:,t+1])
        ext_std[i,8,t] = np.std(l_temp[:,t+1])
        ext_std[i,9,t] = np.std(m_temp[:,t+1])


[      0       1       4 ... 7780833 7780834 7780835]
0 243239
10000 243239
20000 243239
30000 243239
40000 243239
50000 243239
60000 243239
70000 243239
80000 243239
90000 243239
100000 243239
110000 243239
120000 243239
130000 243239
140000 243239
150000 243239
160000 243239
170000 243239
180000 243239
190000 243239
200000 243239
210000 243239
220000 243239
230000 243239
240000 243239


In [14]:
np.save('ext_ubv.npy',ext)
np.save('ext_std_ubv.npy', ext_std)
np.save('parsec_ubv.npy', parsec)
np.save('parsec_std_ubv.npy', parsec_std)

In [15]:
# check that the stds are small and also that Galaxia falls onto those parsec indexes.